## Problem 2

In [1]:
# Imports
import numpy as np
import scipy
import math
import matplotlib.pyplot as plt
import statistics
import pandas as pd

In [2]:
# data
la = 5 # lambda, Demand rate [per day]
mu = 1/3 # arrival rate [per day]
r = 20 # reorder point
Q = 20 # Order quantity
cF = 100 # Fixed cost per order
cH = 2 # Holding cost per item per day
cS = 230 # Stock-out cost per day
L = 1/mu

#### a) 

In [3]:
s0 = 0 #Initial state
A = np.zeros((41, 41))
# Fix 
for i in range(r + 1):
    A[i, i + Q] = mu
# Set demand rate 
for i in range(1, 41):
    A[i, i - 1] = la # ???
# Set diagonals such that each row equals 0
for i in range(41):
    for j in range(41):
        if i == j:
            diag_val = -np.sum(A[i])
            A[i,j] = diag_val
print(f"row 0 = \n{A[0]}")
print(f"row 1 = \n{A[1]}")

print(f"row 20 = \n{A[20]}")
print(f"row 40 = \n{A[-1]}")

row 0 = 
[-0.33333333  0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.33333333  0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.        ]
row 1 = 
[ 5.         -5.33333333  0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.33333333  0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.        ]
row 20 = 
[ 0.          0.          0.          0.          0.          0.
  0

In [4]:
# print("A = ")
# for i in range(41):
#     string = ""
#     for j in range(41):
#         if A[i,j] == 0:
#             string += " 0 "
#         else:
#             string += " " + str(A[i,j])[:3]
#     print(string)
#     print("")
    

#### b) 

To find the steady state solution, we follow the procedure in chapter **7.1.3**. We have the $A$ matrix, and substitute the first column with 1's to get $A_1$. $b$ is simply a column vector with 0s in all places but the first. In matrix form, we have $$P \cdot A_1 = b $$ We transpose both sides of the equation to get $$A_1^T \cdot P^T = b^T$$ This is on standard form, and we can solve it using scipy.linalg.solve.

In [5]:
# Replace first column with 1's in the A-matrix to get A1
A[:,0] = 1
b = np.zeros(41)
b[0] = 1
# b = np.array([b])

# print(At)
PT = scipy.linalg.solve(np.transpose(A), b.T)
P = PT.T
print(P)
# print(f"row 0 = \n{A[0]}")
# print(f"row 20 = \n{A[20]}")
# print(f"row 40 = \n{A[-1]}")

[0.17101476 0.01140098 0.01216105 0.01297179 0.01383657 0.01475901
 0.01574294 0.01679247 0.01791197 0.0191061  0.02037984 0.0217385
 0.02318773 0.02473358 0.02638249 0.02814132 0.03001741 0.03201857
 0.03415314 0.03643002 0.03885868 0.03004828 0.02928821 0.02847748
 0.02761269 0.02669025 0.02570632 0.02465679 0.02353729 0.02234316
 0.02106942 0.01971076 0.01826153 0.01671568 0.01506677 0.01330794
 0.01143185 0.00943069 0.00729612 0.00501925 0.00259058]


In [7]:
ave_time_no_prod = P[0]
print(f"Average time we are not able to produce product 2 = {ave_time_no_prod*100:.2f} % of the time")

Average time we are not able to produce product 2 = 17.10 % of the time


#### c) 

For a stock-out situation to occur, we will have 0 parts left and have a customer that wants a unit. 

In [9]:
stock_out = la*P[0]
print(f"A stock-out situation will occur {stock_out:.2f} [unit???]")

A stock-out situation will occur 0.86 [unit???]


#### d) 

In [14]:
# Usikker på disse, må høre med studass
holding_cost = sum(P*range(41)*cH)
print(f"Average holding cost = {holding_cost:.2f}")
shipping_cost = cF*(la/Q)
print(f"Average shipping cost = {shipping_cost:.2f}")
lost_production_cost = cS*P[0]
print(f"Lost production cost = {lost_production_cost:.2f}")

Average holding cost = 32.54
Average shipping cost = 25.00
Lost production cost = 39.33


#### e) 

Since we order a quantity of Q = 20 as soon as we reach r = 20, that is, in state $P_{20}$, we need to caluclate the probability that four days pass ($4\cdot\lambda$ = 20), given that the arrival rate i $\mu$. Let $F_X$ be the cumulative probability function of the arrival after an order, which is exponentially distributed with mean value $\frac{1}{\mu}$. Then we simply need to multiply the CDF between intervals 4-5 (1 day delay), 5-6 (2 day delay) and so on with the number of days of delay to get the average delay time for stock out situations. To achieve a result, we test for orders up to 100 days

In [28]:
dist = scipy.stats.expon(scale = 1/mu)
FX = lambda d: dist.cdf(d)*(d-4) # d is the number of days
FX(5)
ave_wait_time, err = scipy.integrate.quad(FX, 4, 100)
print(f"Average wait time in stock out situation = {ave_wait_time:.2f}")

Average wait time in stock out situation = 4605.63
